In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import imutils
import numpy as np
import cv2
import os

In [2]:
def detect_mask(frame, faceNet, model):
    # grab the dimensions of the frame and then construct a blob
    (h,w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300,300), (104.0, 177.0, 123.0))
    
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    faces = []
    locs = []
    preds = []
    
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

    # Extract confidence and compare it to the confidence threshold(50%) so that we can filter out detections that are weak.
    # If the algorithm is more than 50% confident that the detection is a face, Iterate over all of the faces detected and extract their start and end points.

        if confidence > 0.5:
            box = detections[0, 0, i, 3:7]*np.array([w,h,w,h])
            (startX, startY, endX, endY) = box.astype('int')

            # ensure the bounding boxes fall within the dimensions of the frame
            startX, startY = (max(0, startX), max(0,startY))
            endX, endY = (min(w-1, endX), min(h-1, endY))

            # extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224,224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))
        

        if len(face) > 0:
            faces = np.array(faces, dtype = 'float32')
            preds = model.predict(faces, batch_size = 10)

        return locs, preds

In [3]:
prototxtPath = os.path.sep.join([r'D:\Deep Learning\FaceMaskDetector\face detector', 'deploy.prototxt'])
weightsPath = os.path.sep.join([r'D:\Deep Learning\FaceMaskDetector\face detector', 'res10_300x300_ssd_iter_140000.caffemodel'])

In [4]:
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [10]:
model = load_model('D:\Deep Learning\FaceMaskDetector\maskdetector.model')

In [11]:
vs = VideoStream(src=0).start()

while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    (locs, preds) = detect_mask(frame, faceNet, model)
    
    for (i,j) in zip(locs, preds):
        (startX, startY, endX, endY) = i
        (withMask, withoutMask) = j
        
        label = "Mask" if withMask > withoutMask else "No Mask"
        color = (0,255,0) if label=='Mask' else (0,0,255)
        
        label="{}: {:.2f}%".format(label,max(withMask,withoutMask)*100)
        
        cv2.putText(frame, label, (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    
    
cv2.destroyAllWindows() 
vs.stop()       